# Quantum Convolutional Neural Network 

## <ins> Convolutional Architecture Building blocks <ins>

In this section, we design and test each part of the QCNN in comparison with PennyLane simulation to ensure the good simulation of the results.

In [13]:
from PennyLane_toolbox import *
from toolbox import *

### <font color='red'>Conv Layer:</font>

We first start with the convolution layer.

In [2]:
from Conv_Layer import *

Let's define a common initial state that we can use to test our Pytorch simulation:

In [3]:
n, k = 8, 2
# Filter size:
K = 4

dev = qml.device("default.qubit", wires=n)

@qml.qnode(dev)
def circuit(angle):
    qml.PauliX(wires=0)
    qml.PauliX(wires=4)
    RBS(0,1, angle)
    RBS(4,5, angle)
    return qml.state()

# We express the final state in the basis of Hamming weight k:
map = map_RBS(n,k)
initial_state = map_Computational_Basis_to_HW_Subspace(n,k,map,torch.tensor(circuit(torch.pi/4)))
initial_rho = torch.einsum('i,j->ij', initial_state, initial_state) # outer product
print("Initial state in the Hamming weight basis:")
print(initial_state)
# We express the final state in the Image basis:
map_I2 = map_RBS_I2_2D(n//2)
print("Initial state in the Image basis:")
initial_state_I2 = map_Computational_Basis_to_Image_Square_Subspace(n,map_I2,torch.tensor(circuit(torch.pi/4)))
initial_rho_I2 = torch.einsum('i,j->ij', initial_state_I2, initial_state_I2)
print(initial_state_I2)

Initial state in the Hamming weight basis:
tensor([ 0.0000,  0.0000,  0.0000,  0.5000, -0.5000,  0.0000,  0.0000,  0.0000,
         0.0000, -0.5000,  0.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000])
Initial state in the Image basis:
tensor([ 0.5000, -0.5000,  0.0000,  0.0000, -0.5000,  0.5000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])


We can now test our convolutional layer <ins>__using Pytorch__<ins>:

In [4]:
# Hardware accelerator:
device = torch.device("cpu")  # Only using CPU

### Pytorch simulation:
CONV = Conv_RBS_state_vector(n//2,K,device)
CONV_I2 = Conv_RBS_state_vector_I2(n//2,K,device)
out_1 = CONV.forward(initial_state.unsqueeze(0)) # Unsqueeze to add batch dimension
out_2 = CONV_I2.forward(initial_state_I2.unsqueeze(0)) # Unsqueeze to add batch dimension
print(out_1)
print(out_2)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0344, -0.0864,  0.1576, -0.0217,  0.0000,
          0.0000, -0.0699,  0.1755, -0.3202,  0.0442,  0.0000,  0.1603, -0.4024,
          0.7343, -0.1013, -0.0557,  0.1397, -0.2550,  0.0352,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000]], grad_fn=<SqueezeBackward1>)
tensor([[ 0.0313, -0.1521, -0.3446,  0.2734,  0.0196, -0.0955, -0.2163,  0.1716,
         -0.0558,  0.2715,  0.6151, -0.4879,  0.0039, -0.0190, -0.0430,  0.0341]],
       grad_fn=<SqueezeBackward1>)


And <ins>__using Pennylane__<ins>:

Considering state simulations:

In [5]:
### PennyLane simulation:
# We extract the parameters from the Pytorch model:
angles = [float(CONV.Parameters[i]) for i in range(len(CONV.Parameters))]
angles_I2 = [float(CONV_I2.Parameters[i]) for i in range(len(CONV_I2.Parameters))]
# We define again the circuit:
Param_dictionary, list_gates = Conv_2D_gates([i for i in range(n)], K)
@qml.qnode(dev)
def Circuit_global(angles):
    qml.PauliX(wires=0)
    qml.PauliX(wires=4)
    RBS(0,1, torch.pi/4)
    RBS(4,5, torch.pi/4)
    Conv_RBS_2D(angles, Param_dictionary, list_gates)
    return qml.state()

print(torch.sum(map_Computational_Basis_to_HW_Subspace(n,k,map,torch.tensor(Circuit_global(angles))) - out_1))
print(torch.sum(map_Computational_Basis_to_Image_Square_Subspace(n,map_I2,torch.tensor(Circuit_global(angles_I2))) - out_2))

tensor(2.7940e-08, grad_fn=<SumBackward0>)
tensor(-1.1455e-07, grad_fn=<SumBackward0>)


Considering density matrix simulation:

In [6]:
### Pytorch simulation:
CONV = Conv_RBS_density(n//2,K,device)
CONV_I2 = Conv_RBS_density_I2(n//2,K,device)
out_1 = CONV.forward(initial_rho.unsqueeze(0)) # Unsqueeze to add batch dimension
out_2 = CONV_I2.forward(initial_rho_I2.unsqueeze(0)) # Unsqueeze to add batch dimension

### PennyLane simulation:
# We extract the parameters from the Pytorch model:
angles = [float(CONV.Parameters[i]) for i in range(len(CONV.Parameters))]
angles_I2 = [float(CONV_I2.Parameters[i]) for i in range(len(CONV_I2.Parameters))]
# We define again the circuit:
Param_dictionary, list_gates = Conv_2D_gates([i for i in range(n)], K)
@qml.qnode(dev)
def Circuit_global_density(angles):
    qml.PauliX(wires=0)
    qml.PauliX(wires=4)
    RBS(0,1, torch.pi/4)
    RBS(4,5, torch.pi/4)
    Conv_RBS_2D(angles, Param_dictionary, list_gates)
    return qml.density_matrix([i for i in range(n)])

print(torch.sum(torch.abs(map_Computational_Basis_to_HW_Subspace_density(n,k,map,torch.tensor(Circuit_global_density(angles)))-out_1)))
print(torch.sum(torch.abs(map_Computational_Basis_to_Image_Square_Subspace_density(n,map_I2,torch.tensor(Circuit_global_density(angles_I2)))-out_2)))

tensor(1.1279e-06, grad_fn=<SumBackward0>)
tensor(9.9256e-07, grad_fn=<SumBackward0>)


### <font color='red'>Dense Layer:</font>

We now study the simulation of the Dense Layer. We will use the same initial state than for the Convolutional layer section.

In [7]:
from Dense import *

In [8]:
# Hardware accelerator:
device = torch.device("cpu")  # Only using CPU

# Dense layer definition:
list_gates = [(i,i+1) for i in range(n-1)]

### Pytorch simulation:
Dense = Dense_RBS_state_vector(n//2,list_gates,device)
Dense_density = Dense_RBS_density(n//2,list_gates,device)

We define the equivalent Pennylane simulation:

In [9]:
@qml.qnode(dev)
def Circuit_global(angles):
    qml.PauliX(wires=0)
    qml.PauliX(wires=4)
    RBS(0,1, torch.pi/4)
    RBS(4,5, torch.pi/4)
    dense_RBS(angles, list_gates)
    return qml.state()

@qml.qnode(dev)
def Circuit_global_density(angles):
    qml.PauliX(wires=0)
    qml.PauliX(wires=4)
    RBS(0,1, torch.pi/4)
    RBS(4,5, torch.pi/4)
    dense_RBS(angles, list_gates)
    return qml.density_matrix([i for i in range(n)])

# We extract the parameters from the Pytorch model:
angles = [float(Dense.RBS_gates[i].angle) for i in range(len(Dense.RBS_gates))]
angles_density = [float(Dense_density.RBS_gates[i].angle) for i in range(len(Dense_density.RBS_gates))]

out_1 = Dense.forward(initial_state.unsqueeze(0)) # Unsqueeze to add batch dimension
out_2 = Dense_density.forward(initial_rho.unsqueeze(0)) # Unsqueeze to add batch dimension
print(torch.sum(torch.abs(map_Computational_Basis_to_HW_Subspace(n,k,map,torch.tensor(Circuit_global(angles))) - out_1)))
print(torch.sum(torch.abs(map_Computational_Basis_to_HW_Subspace_density(n,k,map,torch.tensor(Circuit_global_density(angles_density))) - out_2)))

tensor(1.6880e-07, grad_fn=<SumBackward0>)
tensor(7.6548e-07, grad_fn=<SumBackward0>)


### <font color='red'>Pooling Layer:</font>

We now study the simulation of the Pooling Layer

In [10]:
from Pooling import *

We can now test our pooling layer <ins>__using Pytorch__<ins>:

In [11]:
# Hardware accelerator:
device = torch.device("cpu")  # Only using CPU

### Pytorch simulation:
model = nn.Sequential(Conv_RBS_density_I2(n//2,K,device),Pooling_2D_density(n//2,n//4,device))
out_model = model(initial_rho_I2.unsqueeze(0))
# print(out_model.squeeze())

Let's do the same simulation using PennyLane, showing that the two results are same

In [12]:
def load_parameters(conv):
    angles = [float(conv.Parameters[i]) for i in range(len(conv.Parameters))]
    Param_dictionary, list_gates = Conv_2D_gates([i for i in range(n)], K)
    return angles, Param_dictionary, list_gates

angles, Param_dictionary, list_gates =  load_parameters(model[0])

@qml.qnode(dev)
def Circuit_global(angles):
    qml.PauliX(wires=0)
    qml.PauliX(wires=4)
    RBS(0,1, torch.pi/4)
    RBS(4,5, torch.pi/4)
    Conv_RBS_2D(angles, Param_dictionary, list_gates)
    Pool_2D([i for i in range(n)])
    return qml.density_matrix([1,3,5,7])

new_n = n//2
out_cp = torch.tensor(Circuit_global(angles))
out_simulation = map_Computational_Basis_to_Image_Square_Subspace_density(new_n, map_RBS_I2_2D(new_n//2), out_cp)
print(torch.sum(out_simulation-out_model))

tensor(6.7535e-08, grad_fn=<SumBackward0>)
